In [16]:
import os
import numpy as np
import math
import cv2
import time

In [17]:
def psnr(img1, img2):
    mse = np.mean((img1/1. - img2/1.) ** 2 )
    if mse < 1.0e-10:
        return 100*1.0
    return 10 * math.log10(255.0*255.0/mse)

In [18]:
 def mse(img1,img2):
    mse = np.mean((img1/1. - img2/1.) ** 2 )
    return mse

In [19]:
 def ssim(y_true , y_pred):
    u_true = np.mean(y_true)
    u_pred = np.mean(y_pred)
    var_true = np.var(y_true)
    var_pred = np.var(y_pred)
    std_true = np.sqrt(var_true)
    std_pred = np.sqrt(var_pred)
    c1 = np.square(0.01*7)
    c2 = np.square(0.03*7)
    ssim = (2 * u_true * u_pred + c1) * (2 * std_pred * std_true + c2)
    denom = (u_true ** 2 + u_pred ** 2 + c1) * (var_pred + var_true + c2)
    return ssim / denom

In [20]:
path1 = 'C:/Users/Zheng/Desktop/SR-Note/img_005_SRF_x2.0_SR.tif'  #指定输出结果文件夹
path2 = 'C:/Users/Zheng/Desktop/SR-Note/Original_1024.tif'                #指定原图文件夹

In [22]:
img_a = cv2.imread(path1)
img_b = cv2.imread(path2)
psnr_num = psnr(img_a, img_b)
ssim_num = ssim(img_a, img_b)
mse_num = mse(img_a,img_b)

print("PSNR:",psnr_num)
print("SSIM:",ssim_num)
print("MSE:",mse_num)

PSNR: 23.62210994470502
SSIM: 0.8388434574236953
MSE: 282.4030392964681
